# Combine features

At this stage we are going to combine the behavioural, eeg and mri data.

We are then gonna train a linear model with the purpose to both reconstruct the original data based on a latent space but also based on the predictions for the multi-label problem

For these experiments, we have omitted multiple same diagnoses for the same patient. That means that multiple diagnoses of Neurological diseases for instance will be eliminated.

In [0]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import seaborn as sns
import os
import re

%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = 5, 10
rcParams['font.size'] = 12

plt.style.use('ggplot')

In [3]:
import tensorflow as tf
from collections import Counter
from autoencode_predict import AutoencodePredict
from autoencode_seq import AutoencodeSeq
from autoencode_sequence_predict import AutoencodeSeqPredict
from utils import multi_label_accuracy_precision_recall, f1_per_class, get_predictions_from_sequences
from utils import before_softmax_to_predictions, count_correct, accuracy_per_subject
from visualizations import plot_categories
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from matplotlib import animation
from matplotlib import rc
from IPython.core.display import HTML
rc('animation', html='jshtml')

## Combine features

### Load behavioural data and preprocess dataset

In [4]:
base_dir = '/gdrive/My Drive/Colab Notebooks/DSLab/data'

behaviour_data = pd.read_csv(os.path.join(base_dir, 'HBNFinalSummaries.csv'), low_memory=False)

initial_size = behaviour_data.shape[0]
behaviour_data = behaviour_data[behaviour_data['NoDX'].isin(['Yes', 'No'])]
new_size = behaviour_data.shape[0]
print('Removing', initial_size - new_size,
      'patients as their evaluation was incomplete.')

most_common_disorders = ['Attention-Deficit/Hyperactivity Disorder', 'Anxiety Disorders', 'Specific Learning Disorder',
                         'Autism Spectrum Disorder', 'Disruptive', 'No Diagnosis Given', 'Communication Disorder',
                         'Depressive Disorders']

category_columns = ['DX_' + str(i).zfill(2) + '_Cat' for i in range(1, 11)] +\
                   ['DX_' + str(i).zfill(2) + '_Sub' for i in range(1, 11)]

# find users that have no diagnosis within these top diseases
# filtering should cahnge anything as this should also happen at a later stage
mask = None
for col in category_columns:
    mask_col = behaviour_data[col].isin(most_common_disorders)
    if mask is None:
        mask = mask_col
    else:
        mask = mask | mask_col

initial_size = behaviour_data.shape[0]
behaviour_data = behaviour_data[mask]
behaviour_data = behaviour_data.reset_index(drop=True)
new_size = behaviour_data.shape[0]
print('Removing', initial_size - new_size,
      'patients as their diagnoses were very uncommon.')

Removing 282 patients as their evaluation was incomplete.
Removing 73 patients as their diagnoses were very uncommon.


In [0]:
no_diagnosis_given = 'No Diagnosis Given'

if no_diagnosis_given in most_common_disorders:
    no_diag_index = most_common_disorders.index(no_diagnosis_given)
    most_common_disorders = most_common_disorders[:no_diag_index] + \
        most_common_disorders[no_diag_index + 1:]

In [0]:
diagnoses_to_ids = {disorder: i for i, disorder in enumerate(most_common_disorders)}

In [0]:
def get_disorder(data, row, index):
    disorder = behaviour_data.iloc[row][category_columns[index]]

    if disorder == 'Neurodevelopmental Disorders':
        disorder = behaviour_data.iloc[row][category_columns[index + 10]]

    return disorder

order_of_disorders = []
for k in range(behaviour_data.shape[0]):
    i = 0
    disorder = get_disorder(behaviour_data, k, i)
    disorders_patient = []
    while disorder != no_diagnosis_given and not pd.isnull(disorder):
        if disorder in diagnoses_to_ids:
            if diagnoses_to_ids[disorder] not in disorders_patient:
                disorders_patient.append(diagnoses_to_ids[disorder])
        i += 1
        if i == len(category_columns):
            break
        disorder = get_disorder(behaviour_data, k, i)
    order_of_disorders.append(disorders_patient)

In [0]:
max_len_order = np.max([len(x) for x in order_of_disorders])

# pad with a new token denoting the pad token
pad_token = len(most_common_disorders)
bod_token = len(most_common_disorders) + 1
eod_token = len(most_common_disorders) + 2

order_of_disorders = [[bod_token] + x + [eod_token] + [pad_token] * (max_len_order - len(x)) for x in order_of_disorders]

order_of_disorders = np.array(order_of_disorders)

Many diseases are also present many times. What should we do with them? I have only considered the first presence of a disorder for a patient! 

In [0]:
classes = np.zeros((len(most_common_disorders),
                    behaviour_data.shape[0]), dtype=np.int32)

df_disorders = behaviour_data[category_columns]

for i, disorder in enumerate(most_common_disorders):
    mask = df_disorders.select_dtypes(include=[object]). \
        applymap(lambda x: disorder in x if pd.notnull(x) else False)
    disorder_df = df_disorders[mask.any(axis=1)]
    np.add.at(classes[i], disorder_df.index.values, 1)

Also keep track of the different kinds of ADHD

- F90.0, Attention-deficit hyperactivity disorder, predominantly inattentive type
- F90.1, Attention-deficit hyperactivity disorder, predominantly hyperactive type
- F90.2, Attention-deficit hyperactivity disorder, combined type
- F90.8, Attention-deficit hyperactivity disorder, other type
- F90.9, Attention-deficit hyperactivity disorder, unspecified type

Include errors in type specification ('No Diagnosis Given: Incomplete Eval' and 'nan' to the code F90.9

In [0]:
# only consider the classification of the first ADHD diagnosis
code_columns = ['DX_' + str(i).zfill(2) + '_Code' for i in range(1, 11)]
adhd_categories = np.zeros((len(behaviour_data), 10), dtype=object)

for i in range(10):
    mask = [x is not np.nan and x.startswith('Atten') for x in behaviour_data[category_columns[10 + i]].values]

    # print(behaviour_data[mask][category_columns[10 + i]].values)
    
    for patient_index in range(len(behaviour_data)):
        if mask[patient_index]:
            adhd_categories[patient_index, i] = behaviour_data.iloc[patient_index][code_columns[i]]

adhd_categories_per_patient = ['' for _ in range(len(behaviour_data))]
for patient_index in range(len(behaviour_data)):
    adhd_categories_per_patient[patient_index] = ''.join([x for x in adhd_categories[patient_index] if type(x) == str][:1])

# manually fill at positions where ADHD is knwon but error at code
for i, pred in enumerate(classes[0, :]):
    if pred == 1 and len(adhd_categories_per_patient[i]) == 0:
        adhd_categories_per_patient[i] = 'F90.9'

behaviour_data['adhd_categories'] = adhd_categories_per_patient 

In [0]:
behaviour_data_columns = behaviour_data.columns.values.astype(np.str)

columns_to_drop = behaviour_data_columns[
    np.flatnonzero(np.core.defchararray.find(behaviour_data_columns, 'DX') != -1)]

behaviour_data = behaviour_data.drop(columns=columns_to_drop)

In [0]:
for disorder, classification in zip(most_common_disorders, classes):
    behaviour_data[disorder] = classification

behaviour_data['order_diagnoses'] = list(order_of_disorders)

In [13]:
behaviour_data.shape

(1741, 315)

In [0]:
combined_df = behaviour_data.set_index('EID')

### Patients that have all kind of data

In [0]:
fa_per_tract = pd.read_csv(os.path.join(base_dir, 'FAPerTract.csv'), low_memory=False)
fa_per_tract['ID'] = fa_per_tract['ID'].apply(lambda x: x[:-1] if "/" in x else x)
fa_per_tract = fa_per_tract.drop(columns=['ScanSite'])

cort_thick_l = pd.read_csv(os.path.join(base_dir,'CorticalThicknessLHROI.csv'), low_memory=False)

eeg_mic = pd.read_csv(os.path.join(base_dir, "RestingEEG_Microstates.csv"))
eeg_psd = pd.read_csv(os.path.join(base_dir, "RestingEEG_PSD_Average.csv"))
eeg_spectro = pd.read_csv(os.path.join(base_dir, "RestingEEG_Spectro_Average.csv"))

In [16]:
patients_with_all_data = set(list(fa_per_tract['ID'])) & set(list(combined_df.index)) & set(list(cort_thick_l['ID'])) &\
                            set(eeg_mic['id'].values) & set(eeg_psd['id'].values) & set(eeg_spectro['id'].values)

len(patients_with_all_data)

404

### Load mri and add to dataset

In [0]:
fa_per_tract = pd.read_csv(os.path.join(base_dir, 'FAPerTract.csv'), low_memory=False)

# Remove "/" from the end some IDs 
fa_per_tract['ID'] = fa_per_tract['ID'].apply(lambda x: x[:-1] if "/" in x else x)
fa_per_tract = fa_per_tract.drop(columns=['ScanSite'])

# join with behavioural data
combined_df = combined_df.join(fa_per_tract.set_index('ID'), how='inner')

In [0]:
# base_dir = 'DataScience2019_MRI/MRI/structuralMRI'

# column ScanSite already exists in the behavioural data
cort_thick_l = pd.read_csv(os.path.join(base_dir,
    'CorticalThicknessLHROI.csv'), low_memory=False).drop(columns=['ScanSite'])
cort_thick_r = pd.read_csv(os.path.join(base_dir,
    'CorticalThicknessRHROI.csv'), low_memory=False).drop(columns=['eTIV', 'ScanSite'])
cort_vol_l = pd.read_csv(os.path.join(base_dir,
    'CorticalVolumeLHROI.csv'), low_memory=False).drop(columns=['eTIV', 'ScanSite'])
cort_vol_r = pd.read_csv(os.path.join(base_dir,
    'CorticalVolumeRHROI.csv'), low_memory=False).drop(columns=['eTIV', 'ScanSite'])
sub_cort_vol_l = pd.read_csv(os.path.join(base_dir,
    'SubCorticalVolumeLHROI.csv'), low_memory=False).drop(columns=['eTIV', 'ScanSite'])
sub_cort_vol_r = pd.read_csv(os.path.join(base_dir,
    'SubCorticalVolumeRHROI.csv'), low_memory=False).drop(columns=['eTIV', 'ScanSite'])
glob_thick = pd.read_csv(os.path.join(base_dir,
    'GlobalCorticalThickness.csv'), low_memory=False).drop(columns=['ScanSite'])

# Join tables 
struct_mri = pd.merge(cort_thick_l, cort_thick_r, on='ID', how='inner')
struct_mri = pd.merge(struct_mri, cort_vol_l, on='ID', how='inner')
struct_mri = pd.merge(struct_mri, cort_vol_r, on='ID', how='inner')
struct_mri = pd.merge(struct_mri, sub_cort_vol_l, on='ID', how='inner')
struct_mri = pd.merge(struct_mri, sub_cort_vol_r, on='ID', how='inner')
struct_mri = pd.merge(struct_mri, glob_thick, on='ID', how='inner')

combined_df = combined_df.join(struct_mri.set_index('ID'), how='inner')

In [21]:
combined_df.shape

(747, 709)

### Load EEG  and add to dataset

In [0]:
eeg_mic = pd.read_csv(os.path.join(base_dir, "RestingEEG_Microstates.csv"))
eeg_psd = pd.read_csv(os.path.join(base_dir, "RestingEEG_PSD_Average.csv"))
eeg_spectro = pd.read_csv(os.path.join(base_dir, "RestingEEG_Spectro_Average.csv"))

In [23]:
combined_df = combined_df.join(eeg_mic.set_index('id'), how='inner')
combined_df = combined_df.join(eeg_psd.set_index('id'), how='inner')
combined_df = combined_df.join(eeg_spectro.set_index('id'), how='inner')

combined_df.shape

(404, 1134)

### Some final preprocessing

In [0]:
# fdx and mdx may contain 'No Diagnosis'
# drop them for now but they may be important
# they correspond to father's and mother's primary diagnosis
columns_to_drop = ['Anonymized.ID', 'mdx', 'fdx', 'fcodxm_1', 'fcodxm_2', 'fcodxm_3', 'mcodxm_1',
                   'mcodxm_2', 'mcodxm_3', 'mcodxmdt', 'TOWRE_Total_Desc', 'Picture_Vocab_Raw',
                   'sib1dx', 'sib1codxm_1', 'sib1codxm_2', 'sib1codxm_3',
                   'sib2dx', 'sib2codxm_1', 'sib2codxm_2', 'sib2codxm_3',
                   'sib3dx', 'sib3codxm_1', 'sib3codxm_2', 'sib3codxm_3',
                   'sib4dx', 'sib4codxm_1', 'sib4codxm_2', 'sib4codxm_3',
                   'sib5dx', 'sib5codxm_1', 'sib5codxm_2', 'sib5codxm_3']

combined_df = combined_df.drop(columns=columns_to_drop)

In [0]:
# assert combined_df.shape == (1053, 653) # if you only include the behavioural and the structural

classes = combined_df[most_common_disorders].values
orders = np.stack(combined_df['order_diagnoses'].values)
adhd_categories = combined_df['adhd_categories'].values

combined_df = combined_df.drop(columns=most_common_disorders + ['order_diagnoses'] + ['adhd_categories'])

In [26]:
np.sum(classes, axis=0) / len(classes)

array([0.58415842, 0.26732673, 0.15841584, 0.14108911, 0.15841584,
       0.11138614, 0.11386139])

## Filter to noly users that have all kind of data

In [27]:
train_only_intersection = False

mask = combined_df.index.isin(patients_with_all_data)

if train_only_intersection:
    print('ONLY USING INTERSECTION DATA!')

    combined_df = combined_df[mask]
    classes = classes[mask]
    orders = orders[mask]
else:
    print('TRAIN ON ALL AVAILABLE DATA!')
    
    combined_df_intersection = combined_df[mask]
    classes_intersection = classes[mask]
    orders_intersection = orders[mask]

TRAIN ON ALL AVAILABLE DATA!


## Cross validation

In [0]:
import pickle

# load the same cross validation splits to have a common ground for all classifiers and all methods
with open('cross_validation_splits.pkl', 'rb') as fid:
    cross_validation_splits = pickle.load(fid)

In [0]:
def mean_imputer(x, y):
    return np.where(np.isnan(x), np.ma.array(x, mask=np.isnan(x)).mean(axis=0), x),\
            np.where(np.isnan(y), np.ma.array(x, mask=np.isnan(x)).mean(axis=0), y)  

def process_dataset(x, cols):
    x1 = x.drop(columns=cols).isna().astype('int32')

    # change column naming to enable the inner join
    x1.columns = [col + '_existence' for col in x1.columns]
    return x.join(x1)

def data_and_existence_of_features(x, y):
    cols = ['Sex', 'Age', 'Study.Site']
    rest_of_columns = list(x.columns.values)
    
    for c in cols:
        rest_of_columns.remove(c)

    x_new = process_dataset(x, cols)
    y_new = process_dataset(y, cols)
    
    return x_new, y_new

def run_cross_validation(df, classes, orders, fit_model_and_get_predictions, cross_validation_splits,
                         drop_missing_threshold=0.5, add_existence_of_features=False):

    flag = True
    f1s = []

    for i, cross_validation in enumerate(cross_validation_splits):

        final_predictions = np.zeros((len(patients_with_all_data), classes.shape[1]), dtype=np.int16)

        for test_ids in cross_validation:

            mask = df.index.isin(test_ids)

            if train_only_intersection:
                test_index = np.where(mask == True)[0]
            else:
                mask_intersection = combined_df_intersection.index.isin(test_ids)
                test_index = np.where(mask_intersection == True)[0]

        
            train, test = df[~mask], df[mask]

            train_classes, test_classes = classes[~mask], classes[mask]
            train_orders, test_orders = orders[~mask], orders[mask]

            if flag:
                print('Train size', len(train), 'test size', len(test))
                print('Train size', len(train_classes), 'test size', len(test_classes))
                flag = False

            columns_mask = pd.isnull(train).sum() / train.shape[0] >= drop_missing_threshold

            print('Droping this many columns:', np.sum(columns_mask))

            dropped_columns = train.columns[columns_mask]

            train = train.drop(columns=dropped_columns)
            test = test.drop(columns=dropped_columns)

            if add_existence_of_features:
                train, test = data_and_existence_of_features(train, test)

            train_mask = pd.isnull(train).values.astype(np.float32)
            test_mask = pd.isnull(test).values.astype(np.float32)

            # deal with numpy because of some weird SettingWithCopyWarning I cannot figure out
            # impute based on the mean values
            train, test = mean_imputer(train.values, test.values)

            scaler = MinMaxScaler()
            # scaler = StandardScaler()
            train = scaler.fit_transform(train)
            test = scaler.transform(test)

            final_predictions[test_index] = fit_model_and_get_predictions(\
                        train, train_mask, train_classes, train_orders, test, test_mask, test_classes, test_orders)

        if train_only_intersection:
            f1s.append(evaluate_predictions(classes, final_predictions, None, None))
        else:
            f1s.append(evaluate_predictions(classes_intersection, final_predictions, None, None))

    return f1s

def evaluate_predictions(classes, pred_classes, orders, pred_orders):
    ret = None
    if pred_classes is not None:
        print('Pred classes statistics')
        print('multi_label_metrics', multi_label_accuracy_precision_recall(classes, pred_classes))
        print('f1_per_class', f1_per_class(classes, pred_classes))
        print('accuracy_per_subject', accuracy_per_subject(classes, pred_classes))
        ret = f1_per_class(classes, pred_classes)
    if pred_orders is not None:
        print('Pred orders statistics')
        predictions_from_sequences = get_predictions_from_sequences(pred_orders, len(most_common_disorders))
        print('multi_label_metrics', multi_label_accuracy_precision_recall(classes, predictions_from_sequences))
        print('f1_per_class', f1_per_class(classes, predictions_from_sequences))
        print('accuracy_per_subject', accuracy_per_subject(classes, predictions_from_sequences))
        if ret is None:
            ret = f1_per_class(classes, predictions_from_sequences)

    return ret

### Lets see how our method holds againt a simple classical classifier

In [0]:
# clf is global
def classical_classification(train, train_mask, train_classes, train_orders, 
                             test, test_mask, test_classes, test_orders):
    
    predictions = np.zeros((test.shape[0], train_classes.shape[1]), dtype=np.int16)

    for i in range(train_classes.shape[1]):
        ###
        # temp
        # from sklearn.feature_selection import SelectKBest, chi2
        # selector = SelectKBest(chi2, k=k_selection)
        # train = selector.fit_transform(train, train_classes[:, i])
        # test = selector.transform(test)
        # ###

        clf.fit(train, train_classes[:, i])
        predictions[:, i] = clf.predict(test)

    return predictions

In [0]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

f1s = []
thres = 1
ex = False

params = (thres, ex)
print(params)
# clf = LogisticRegression(class_weight='balanced', solver='lbfgs', max_iter=500)
# clf = SVC(class_weight='balanced', C=1, gamma='scale')
clf = RandomForestClassifier(class_weight='balanced', n_estimators=100, max_depth=4)
f1s = run_cross_validation(combined_df, classes, orders, classical_classification, 
                        cross_validation_splits, drop_missing_threshold=thres, add_existence_of_features=ex)

In [0]:
np.mean(f1s, axis=0)

### Autoencoder architectures

In [0]:
def autoencode_predict_fit_predict(train, train_mask, train_classes, train_orders, 
                                   test, test_mask, test_classes, test_orders):
    alpha=0.2          # parameter showing the significance of the prediction loss
    activation=tf.nn.relu
    layers=[80, 20]
    prediction_layers=[50]
    dropout=0.1
    regularization=1e-5
    masking=0.1

    model = AutoencodePredict(train.shape[1], len(most_common_disorders), alpha=alpha, activation=activation, layers=layers,
                            prediction_layers=prediction_layers, dropout=dropout, regularization=regularization, 
                            masking=masking)    

    learning_rate = 3e-3

    def smoothing_fun(a, beta=1):
        return [min(i, 1) for i in a] + beta * np.log([max(i - 1, 1) for i in a])

    pos_weights = smoothing_fun(1 / (np.sum(train_classes, axis=0) / train_classes.shape[0]))
    class_weights = [1] * train_classes.shape[1]

    decay_steps = train.shape[0] // 64 * 5
    
    ensemble_size = 10

    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    # ensemble
    final_predictions = np.zeros((ensemble_size, test.shape[0], len(most_common_disorders)))

    for i in range(ensemble_size):
        model.fit(train, train_mask, train_classes, test_data=test, test_data_mask=test_mask, test_data_labels=test_classes,
                n_epochs=301, print_every_epochs=50, pos_weights=pos_weights, class_weights=class_weights,
                decay_steps=decay_steps)
            
        final_predictions[i] = model.predict(test, make_integer=False)


    # possible combinations of predictions
    # before_softmax_to_predictions(np.mean(final_predictions, axis=0))
    # before_softmax_to_predictions(np.median(final_predictions, axis=0))
    # (np.mean(sigmoid(final_predictions), axis=0) > 0.5).astype(np.int)
    
    # this model cannot predict the order
    return (np.mean(sigmoid(final_predictions), axis=0) > 0.5).astype(np.int)

# f1s = []
# for thres in np.arange(0.1, 1.1, 0.1):
    # for ex in [True, False]:
thres = 0.5
ex = False
params = (thres, ex)
print(params)
f1s = run_cross_validation(combined_df, classes, orders, autoencode_predict_fit_predict, cross_validation_splits)

np.mean(f1s, axis=0)

### Final plots

In [0]:
plot_only_data_from_intersection = True

classifiers = ['SVM', 'RF', 'AE']
data_configurations = ['BEH', 'BEH+MRI', 'BEH+EEG', 'ALL']
number_of_cross_val_runs = 10
# size (different_data_configurations, number_of_classifiers, cross_val_runs, number_of_disorders)
if plot_only_data_from_intersection:
    data = \
    [[[[0.7766179540709812, 0.5140562248995983, 0.48554913294797697, 0.4307692307692308, 0.5316455696202532, 0.3461538461538462, 0.46956521739130436], [0.7863247863247863, 0.5511811023622046, 0.47904191616766467, 0.4393939393939394, 0.5590062111801242, 0.3793103448275862, 0.4552845528455285], [0.7584745762711863, 0.5622489959839357, 0.4772727272727273, 0.4, 0.5454545454545454, 0.3214285714285714, 0.49572649572649574], [0.7807933194154489, 0.5403225806451613, 0.47252747252747257, 0.44776119402985076, 0.5662650602409639, 0.3448275862068965, 0.4137931034482759], [0.7688984881209502, 0.5403225806451613, 0.5365853658536586, 0.4029850746268657, 0.5732484076433121, 0.3571428571428571, 0.4915254237288136], [0.7553648068669528, 0.5381526104417671, 0.5421686746987951, 0.4328358208955224, 0.5093167701863354, 0.34862385321100914, 0.47863247863247865], [0.7872340425531915, 0.5425101214574899, 0.4795321637426901, 0.4444444444444444, 0.5095541401273884, 0.3364485981308411, 0.5043478260869565], [0.7761194029850746, 0.5396825396825397, 0.48780487804878053, 0.42187499999999994, 0.5949367088607594, 0.3333333333333333, 0.47787610619469034], [0.7780126849894291, 0.5375494071146245, 0.46060606060606063, 0.39097744360902253, 0.5384615384615383, 0.2935779816513761, 0.4915254237288136], [0.7805907172995781, 0.5203252032520325, 0.4938271604938272, 0.40624999999999994, 0.5030674846625767, 0.34285714285714286, 0.49572649572649574]],
      [[0.8050314465408805, 0.4392523364485982, 0.2962962962962963, 0.3333333333333333, 0.48120300751879697, 0.1090909090909091, 0.46315789473684216], [0.7876288659793814, 0.4807692307692307, 0.3333333333333333, 0.40740740740740744, 0.4838709677419355, 0.24137931034482757, 0.37894736842105264], [0.8041666666666666, 0.4780487804878049, 0.32142857142857145, 0.38596491228070173, 0.49230769230769234, 0.13793103448275862, 0.3870967741935484], [0.8115942028985509, 0.4174757281553398, 0.297029702970297, 0.4952380952380952, 0.512, 0.0, 0.4285714285714286], [0.8008474576271186, 0.49074074074074076, 0.3177570093457944, 0.41818181818181815, 0.5581395348837209, 0.1818181818181818, 0.4791666666666667], [0.8024439918533605, 0.4711538461538462, 0.22222222222222224, 0.42990654205607476, 0.556390977443609, 0.1090909090909091, 0.43010752688172044], [0.8066528066528066, 0.47058823529411764, 0.29411764705882354, 0.40740740740740744, 0.5354330708661418, 0.11320754716981134, 0.43010752688172044], [0.7966804979253113, 0.48372093023255813, 0.3392857142857143, 0.45045045045045046, 0.5190839694656488, 0.1509433962264151, 0.45161290322580644], [0.8208333333333332, 0.5140186915887851, 0.34285714285714286, 0.4380952380952381, 0.515625, 0.07407407407407407, 0.43749999999999994], [0.8075313807531381, 0.48543689320388356, 0.39252336448598124, 0.36893203883495146, 0.5, 0.2456140350877193, 0.43749999999999994]],
      [[0.7651821862348178, 0.540084388185654, 0.37288135593220334, 0.45, 0.5285714285714286, 0.26506024096385544, 0.5094339622641509], [0.7540983606557377, 0.5762711864406779, 0.38709677419354843, 0.4166666666666667, 0.5379310344827586, 0.3291139240506329, 0.5046728971962617], [0.7559999999999999, 0.5579399141630902, 0.37288135593220334, 0.4918032786885246, 0.5571428571428572, 0.367816091954023, 0.48], [0.769857433808554, 0.5508474576271187, 0.3934426229508197, 0.4406779661016949, 0.5034965034965034, 0.20253164556962028, 0.4528301886792453], [0.7428571428571428, 0.5258620689655172, 0.3448275862068966, 0.46153846153846156, 0.5616438356164384, 0.3999999999999999, 0.5242718446601942], [0.7639999999999999, 0.5630252100840336, 0.3859649122807018, 0.45378151260504196, 0.5211267605633803, 0.31707317073170727, 0.45098039215686275], [0.7780040733197556, 0.5439330543933054, 0.4262295081967213, 0.4705882352941176, 0.547945205479452, 0.3333333333333333, 0.4999999999999999], [0.7667342799188641, 0.5643153526970954, 0.3272727272727273, 0.4444444444444445, 0.547945205479452, 0.37974683544303794, 0.4672897196261682], [0.7525150905432595, 0.5643153526970954, 0.3793103448275862, 0.4424778761061947, 0.5314685314685316, 0.28205128205128205, 0.4999999999999999], [0.7670682730923695, 0.5454545454545454, 0.39252336448598124, 0.45, 0.5517241379310345, 0.325, 0.49090909090909096]]],
     [[[0.7669491525423728, 0.42918454935622324, 0.40875912408759124, 0.32989690721649484, 0.42748091603053434, 0.208955223880597, 0.48275862068965514], [0.7780126849894291, 0.4727272727272727, 0.4029850746268657, 0.3106796116504854, 0.48062015503875966, 0.15625, 0.5161290322580645], [0.7542372881355932, 0.5155555555555555, 0.3458646616541354, 0.23404255319148934, 0.4603174603174603, 0.17910447761194032, 0.4444444444444445], [0.7599164926931107, 0.4736842105263158, 0.3941605839416058, 0.28865979381443296, 0.49206349206349204, 0.09677419354838708, 0.5176470588235293], [0.7420042643923241, 0.497737556561086, 0.3913043478260869, 0.31372549019607837, 0.46511627906976744, 0.18750000000000003, 0.5], [0.7644628099173554, 0.4642857142857143, 0.4233576642335766, 0.31683168316831684, 0.48484848484848486, 0.15625, 0.5], [0.7916666666666666, 0.46017699115044247, 0.3731343283582089, 0.3725490196078431, 0.4360902255639098, 0.21538461538461537, 0.48936170212765956], [0.7542372881355932, 0.502127659574468, 0.37681159420289856, 0.2916666666666667, 0.5396825396825397, 0.18181818181818182, 0.49438202247191015], [0.7656903765690377, 0.48401826484018273, 0.3731343283582089, 0.3106796116504854, 0.4732824427480916, 0.16129032258064516, 0.47191011235955055], [0.7594936708860759, 0.4931506849315069, 0.3880597014925374, 0.2947368421052632, 0.5, 0.19672131147540986, 0.45161290322580644]],
      [[0.7894736842105263, 0.39080459770114945, 0.0547945205479452, 0.0625, 0.34615384615384615, 0.0, 0.2727272727272727], [0.7885010266940452, 0.3626373626373627, 0.05970149253731343, 0.14705882352941174, 0.32, 0.0, 0.2985074626865672], [0.7852760736196318, 0.32142857142857145, 0.05555555555555555, 0.125, 0.3619047619047619, 0.0, 0.33333333333333337], [0.794238683127572, 0.41420118343195267, 0.08450704225352113, 0.09836065573770492, 0.38461538461538464, 0.0, 0.3611111111111111], [0.7835051546391752, 0.46739130434782605, 0.05555555555555555, 0.09375, 0.30612244897959184, 0.0, 0.4], [0.7807933194154489, 0.38372093023255816, 0.08108108108108107, 0.14925373134328357, 0.34615384615384615, 0.0, 0.3478260869565218], [0.7817047817047817, 0.37894736842105264, 0.08333333333333333, 0.20289855072463764, 0.2857142857142857, 0.0, 0.4], [0.7917525773195876, 0.40659340659340665, 0.08108108108108107, 0.12307692307692307, 0.33999999999999997, 0.0, 0.36619718309859156], [0.7866108786610878, 0.38674033149171266, 0.12820512820512822, 0.2631578947368421, 0.35294117647058826, 0.0, 0.3714285714285714], [0.7901234567901235, 0.39999999999999997, 0.08333333333333333, 0.11111111111111112, 0.2978723404255319, 0.0, 0.40540540540540543]],
      [[0.7379454926624739, 0.5240174672489083, 0.2653061224489796, 0.4137931034482759, 0.4999999999999999, 0.13333333333333333, 0.4123711340206186], [0.7407407407407408, 0.5520361990950227, 0.29411764705882354, 0.4339622641509434, 0.4861111111111111, 0.2105263157894737, 0.4680851063829787], [0.744466800804829, 0.5526315789473685, 0.21782178217821785, 0.3888888888888889, 0.5223880597014925, 0.27397260273972607, 0.4313725490196078], [0.7541666666666667, 0.543859649122807, 0.28037383177570097, 0.43636363636363634, 0.5333333333333333, 0.2105263157894737, 0.4693877551020408], [0.7257731958762886, 0.5240174672489083, 0.22916666666666666, 0.42105263157894735, 0.4696969696969697, 0.26229508196721313, 0.4693877551020408], [0.7449392712550608, 0.5454545454545454, 0.2772277227722772, 0.3669724770642202, 0.524822695035461, 0.23333333333333334, 0.4897959183673469], [0.7520661157024793, 0.5232067510548524, 0.25490196078431376, 0.4036697247706422, 0.47058823529411764, 0.19672131147540986, 0.49484536082474223], [0.7505154639175258, 0.5627705627705627, 0.2745098039215686, 0.44036697247706424, 0.5037037037037037, 0.2950819672131148, 0.4742268041237114], [0.7336065573770492, 0.5309734513274337, 0.29126213592233013, 0.4, 0.4852941176470589, 0.25396825396825395, 0.4742268041237114], [0.7351129363449691, 0.5248868778280543, 0.32323232323232326, 0.39622641509433965, 0.5333333333333333, 0.2295081967213115, 0.47619047619047616]]],
     [[[0.7763713080168777, 0.5158730158730159, 0.4680851063829787, 0.3484848484848485, 0.47500000000000003, 0.31372549019607837, 0.46874999999999994], [0.7803837953091683, 0.5019920318725101, 0.39106145251396646, 0.3428571428571428, 0.45679012345679015, 0.3818181818181818, 0.46399999999999997], [0.7639484978540771, 0.5081967213114753, 0.42857142857142855, 0.36507936507936506, 0.47133757961783435, 0.27450980392156865, 0.4065040650406504], [0.7695560253699789, 0.5020576131687243, 0.40659340659340654, 0.40310077519379844, 0.440251572327044, 0.2830188679245283, 0.46551724137931033], [0.7494553376906318, 0.48535564853556495, 0.4861878453038673, 0.3875968992248062, 0.4761904761904762, 0.33333333333333337, 0.4615384615384615], [0.7669491525423728, 0.5490196078431373, 0.42045454545454547, 0.3188405797101449, 0.47133757961783435, 0.2828282828282828, 0.49599999999999994], [0.773218142548596, 0.5161290322580645, 0.47252747252747257, 0.4142857142857143, 0.45, 0.33999999999999997, 0.46399999999999997], [0.7728237791932059, 0.532258064516129, 0.45901639344262296, 0.37956204379562036, 0.5256410256410257, 0.297029702970297, 0.46774193548387094], [0.7831578947368422, 0.515625, 0.3236994219653179, 0.4, 0.47368421052631576, 0.2795698924731182, 0.453781512605042], [0.7737843551797041, 0.5158730158730159, 0.41530054644808745, 0.4122137404580153, 0.4727272727272727, 0.29906542056074764, 0.46031746031746035]],
      [[0.7817047817047817, 0.37234042553191493, 0.16470588235294117, 0.1794871794871795, 0.43636363636363634, 0.0, 0.358974358974359], [0.7725321888412017, 0.3979057591623037, 0.16, 0.13157894736842105, 0.4074074074074074, 0.0425531914893617, 0.3561643835616438], [0.7747368421052633, 0.41530054644808745, 0.1839080459770115, 0.11111111111111112, 0.3818181818181817, 0.0, 0.22535211267605634], [0.7958762886597938, 0.3615819209039548, 0.1951219512195122, 0.20224719101123595, 0.39999999999999997, 0.0, 0.3768115942028985], [0.782051282051282, 0.39999999999999997, 0.17283950617283952, 0.16, 0.3272727272727273, 0.0, 0.3888888888888889], [0.7807933194154489, 0.4193548387096774, 0.1518987341772152, 0.17142857142857143, 0.48214285714285715, 0.0392156862745098, 0.3835616438356165], [0.7782426778242678, 0.37569060773480667, 0.1794871794871795, 0.21951219512195122, 0.36363636363636365, 0.0, 0.2857142857142857], [0.7721518987341772, 0.35294117647058826, 0.17073170731707318, 0.2857142857142857, 0.4485981308411215, 0.0784313725490196, 0.3714285714285714], [0.7617021276595745, 0.37988826815642457, 0.2298850574712644, 0.23684210526315788, 0.45217391304347826, 0.04166666666666667, 0.1971830985915493], [0.7653276955602537, 0.33333333333333337, 0.2823529411764706, 0.20512820512820512, 0.45045045045045046, 0.0425531914893617, 0.43037974683544306]],
      [[0.7648261758691206, 0.5429864253393665, 0.3423423423423423, 0.45, 0.5037037037037037, 0.21333333333333337, 0.48598130841121495], [0.7740585774058578, 0.5565217391304348, 0.3697478991596639, 0.42105263157894735, 0.4931506849315069, 0.30769230769230765, 0.5046728971962617], [0.7535641547861507, 0.5610859728506787, 0.3423423423423423, 0.45614035087719296, 0.5035971223021581, 0.25, 0.4220183486238532], [0.7755102040816325, 0.509090909090909, 0.3448275862068966, 0.4424778761061947, 0.41726618705035967, 0.2028985507246377, 0.44000000000000006], [0.7586206896551725, 0.5044247787610621, 0.3148148148148148, 0.4754098360655738, 0.4929577464788733, 0.31168831168831174, 0.4752475247524752], [0.7617107942973522, 0.5367965367965367, 0.28846153846153844, 0.4375, 0.4657534246575343, 0.2564102564102564, 0.4807692307692307], [0.7901234567901235, 0.5217391304347827, 0.37735849056603776, 0.5040650406504066, 0.49635036496350365, 0.2191780821917808, 0.5046728971962617], [0.7701863354037267, 0.5258620689655172, 0.2830188679245283, 0.43902439024390244, 0.4827586206896552, 0.28205128205128205, 0.4571428571428572], [0.7500000000000001, 0.5502183406113538, 0.2882882882882883, 0.4918032786885246, 0.4748201438848921, 0.2058823529411765, 0.48543689320388345], [0.7463917525773196, 0.5178571428571428, 0.2882882882882883, 0.4406779661016949, 0.43055555555555564, 0.25, 0.5]]],
     [[[0.7619047619047619, 0.4634146341463415, 0.4347826086956521, 0.33333333333333337, 0.4055944055944056, 0.20779220779220778, 0.4554455445544554], [0.7619047619047619, 0.46808510638297873, 0.3878787878787879, 0.2476190476190476, 0.4460431654676259, 0.17500000000000002, 0.4571428571428572], [0.735930735930736, 0.5145228215767634, 0.35802469135802467, 0.25242718446601936, 0.43795620437956206, 0.17283950617283952, 0.43564356435643564], [0.7304347826086958, 0.4491525423728814, 0.43373493975903615, 0.3392857142857143, 0.4647887323943662, 0.24390243902439024, 0.44000000000000006], [0.7461368653421633, 0.46808510638297873, 0.4444444444444444, 0.37837837837837834, 0.4195804195804196, 0.23255813953488372, 0.4380952380952381], [0.7319148936170214, 0.4773662551440329, 0.4, 0.3247863247863248, 0.4460431654676259, 0.21176470588235294, 0.4901960784313726], [0.7473002159827216, 0.4396551724137931, 0.4125, 0.36666666666666664, 0.4316546762589928, 0.24390243902439024, 0.47619047619047616], [0.7531914893617021, 0.47500000000000003, 0.3928571428571428, 0.3185840707964602, 0.49635036496350365, 0.20779220779220778, 0.4752475247524752], [0.7516198704103673, 0.4890829694323144, 0.3544303797468354, 0.3243243243243243, 0.4028776978417266, 0.2285714285714286, 0.4444444444444445], [0.7450980392156863, 0.48535564853556495, 0.4177215189873418, 0.39655172413793105, 0.4689655172413793, 0.20253164556962028, 0.46601941747572817]],
      [[0.7824267782426778, 0.32941176470588235, 0.0, 0.09523809523809525, 0.3125, 0.0, 0.3428571428571428], [0.7682672233820459, 0.3580246913580248, 0.057971014492753624, 0.03278688524590164, 0.29213483146067415, 0.0, 0.25806451612903225], [0.7795918367346939, 0.378698224852071, 0.02857142857142857, 0.06666666666666667, 0.21739130434782608, 0.0, 0.26666666666666666], [0.76890756302521, 0.3832335329341317, 0.056338028169014086, 0.09090909090909091, 0.1627906976744186, 0.0, 0.411764705882353], [0.7653276955602537, 0.4382022471910112, 0.0, 0.12307692307692307, 0.303030303030303, 0.0, 0.25], [0.7784679089026915, 0.4252873563218391, 0.056338028169014086, 0.03225806451612903, 0.30769230769230765, 0.0, 0.22950819672131148], [0.7617021276595745, 0.3431952662721894, 0.057971014492753624, 0.031746031746031744, 0.26086956521739124, 0.0, 0.3125], [0.7818930041152264, 0.3333333333333333, 0.0, 0.03125, 0.4175824175824176, 0.0, 0.26666666666666666], [0.7708333333333334, 0.360248447204969, 0.08333333333333333, 0.03278688524590164, 0.3225806451612903, 0.0, 0.3880597014925373], [0.7644628099173554, 0.3764705882352941, 0.06060606060606061, 0.14925373134328357, 0.33707865168539325, 0.0, 0.3529411764705882]],
      [[0.7484662576687117, 0.5175438596491228, 0.29126213592233013, 0.4, 0.43939393939393934, 0.1764705882352941, 0.43010752688172044], [0.7667342799188641, 0.5155555555555555, 0.26262626262626265, 0.3364485981308411, 0.44274809160305345, 0.23333333333333334, 0.4554455445544554], [0.7484662576687117, 0.5066666666666667, 0.24074074074074076, 0.29411764705882354, 0.5, 0.12903225806451613, 0.4615384615384615], [0.7651821862348178, 0.47441860465116276, 0.3018867924528302, 0.34285714285714286, 0.43939393939393934, 0.13793103448275862, 0.45161290322580644], [0.7287784679089027, 0.5066666666666667, 0.23655913978494625, 0.44036697247706424, 0.42519685039370075, 0.22580645161290322, 0.46315789473684216], [0.7540322580645161, 0.5225225225225226, 0.21052631578947367, 0.34862385321100914, 0.43478260869565216, 0.21875, 0.4693877551020408], [0.7695473251028805, 0.5221238938053098, 0.2653061224489796, 0.3669724770642202, 0.44274809160305345, 0.2, 0.4752475247524752], [0.7663934426229508, 0.49557522123893805, 0.26804123711340205, 0.3269230769230769, 0.4592592592592592, 0.2903225806451613, 0.4444444444444445], [0.7433264887063655, 0.5066666666666667, 0.19230769230769232, 0.30769230769230765, 0.5185185185185185, 0.16129032258064516, 0.4693877551020408], [0.7351129363449691, 0.4811320754716981, 0.3137254901960784, 0.34951456310679613, 0.43939393939393934, 0.21538461538461537, 0.49504950495049505]]]]

else:
    data = \
    [[[[0.7999999999999999, 0.5772357723577235, 0.49723756906077354, 0.5263157894736842, 0.5714285714285714, 0.3466666666666667, 0.5000000000000001], [0.8026315789473684, 0.5795918367346939, 0.4835164835164835, 0.5454545454545454, 0.5838509316770186, 0.381578947368421, 0.48854961832061067], [0.8070175438596492, 0.5877551020408163, 0.5136612021857924, 0.5263157894736842, 0.5548387096774193, 0.3684210526315789, 0.5190839694656488], [0.8008849557522124, 0.5819672131147542, 0.5026737967914439, 0.5562913907284768, 0.5875, 0.3790849673202615, 0.523076923076923], [0.7919463087248323, 0.5819672131147542, 0.5274725274725275, 0.535031847133758, 0.5838509316770186, 0.348993288590604, 0.49624060150375937], [0.8, 0.5587044534412955, 0.4945054945054946, 0.5290322580645161, 0.5660377358490566, 0.3355704697986577, 0.523076923076923], [0.8026315789473684, 0.582995951417004, 0.48587570621468934, 0.535031847133758, 0.5714285714285714, 0.3684210526315789, 0.49624060150375937], [0.8097345132743363, 0.5748987854251013, 0.5108695652173912, 0.5454545454545454, 0.5897435897435898, 0.3513513513513513, 0.515625], [0.8043956043956043, 0.5680000000000001, 0.5, 0.5490196078431372, 0.5897435897435898, 0.36129032258064514, 0.5116279069767442], [0.7893569844789357, 0.5785123966942148, 0.4971751412429378, 0.5695364238410595, 0.5838509316770186, 0.3708609271523179, 0.5076923076923077]],
      [[0.7692307692307693, 0.5533596837944663, 0.4239130434782609, 0.4534883720930233, 0.5485714285714286, 0.34782608695652173, 0.4963503649635036], [0.7789934354485777, 0.5511811023622046, 0.41884816753926696, 0.4685714285714286, 0.5614035087719298, 0.34965034965034963, 0.5035971223021583], [0.7654867256637168, 0.5577689243027888, 0.41111111111111115, 0.47674418604651164, 0.5263157894736842, 0.30985915492957744, 0.5], [0.7750556792873051, 0.5625, 0.4444444444444444, 0.4705882352941176, 0.5647058823529413, 0.2937062937062937, 0.5034965034965035], [0.7723214285714287, 0.552, 0.4375, 0.45086705202312144, 0.5465116279069767, 0.3188405797101449, 0.48920863309352525], [0.7555555555555554, 0.5657370517928287, 0.4692737430167597, 0.4597701149425287, 0.5536723163841808, 0.30344827586206896, 0.46808510638297873], [0.7816593886462881, 0.5433070866141733, 0.43076923076923074, 0.47337278106508873, 0.5348837209302325, 0.32592592592592595, 0.47058823529411764], [0.775330396475771, 0.5511811023622046, 0.44329896907216493, 0.46590909090909083, 0.5529411764705883, 0.3262411347517731, 0.48951048951048953], [0.7616926503340757, 0.5447470817120622, 0.4680851063829787, 0.48554913294797686, 0.5465116279069767, 0.3333333333333333, 0.4788732394366197], [0.7682119205298013, 0.5714285714285715, 0.4835164835164835, 0.4470588235294118, 0.5402298850574713, 0.3356643356643357, 0.4963503649635036]],
      [[0.8232931726907631, 0.6018518518518519, 0.48120300751879697, 0.5820895522388059, 0.6081081081081082, 0.2921348314606741, 0.5585585585585586], [0.8159999999999998, 0.5963302752293578, 0.44274809160305345, 0.5538461538461539, 0.6184210526315789, 0.2758620689655172, 0.5309734513274337], [0.8096192384769539, 0.6009389671361502, 0.45454545454545453, 0.5864661654135339, 0.5921052631578948, 0.2278481012658228, 0.5636363636363637], [0.8178137651821863, 0.6046511627906976, 0.4375, 0.5757575757575758, 0.6027397260273972, 0.2727272727272727, 0.5405405405405406], [0.8096192384769539, 0.5936073059360731, 0.4477611940298507, 0.5757575757575758, 0.6027397260273972, 0.3333333333333333, 0.547008547008547], [0.8143712574850299, 0.5990783410138248, 0.44274809160305345, 0.556390977443609, 0.5733333333333334, 0.2758620689655172, 0.5641025641025642], [0.8286852589641435, 0.5963302752293578, 0.44274809160305345, 0.5714285714285715, 0.6111111111111112, 0.2619047619047619, 0.5535714285714286], [0.8240000000000001, 0.5882352941176471, 0.453125, 0.5671641791044776, 0.6027397260273972, 0.21686746987951808, 0.5765765765765766], [0.8296593186372745, 0.5674418604651164, 0.43939393939393934, 0.5777777777777778, 0.6174496644295302, 0.28571428571428575, 0.5688073394495413], [0.8145161290322581, 0.5779816513761468, 0.4496124031007752, 0.5820895522388059, 0.6206896551724138, 0.28571428571428575, 0.5391304347826086]]],
     [[[0.7219730941704036, 0.5151515151515151, 0.35121951219512193, 0.38095238095238093, 0.49473684210526314, 0.22754491017964074, 0.4230769230769231], [0.7174887892376681, 0.5597014925373134, 0.35643564356435636, 0.4, 0.5133689839572194, 0.2261904761904762, 0.44594594594594594], [0.7207207207207208, 0.5468164794007491, 0.3414634146341463, 0.3978494623655914, 0.5052631578947369, 0.26900584795321636, 0.4473684210526316], [0.7260579064587973, 0.5267175572519084, 0.3529411764705882, 0.40659340659340654, 0.5235602094240838, 0.25301204819277107, 0.4635761589403974], [0.7207207207207208, 0.5454545454545455, 0.3414634146341463, 0.3829787234042553, 0.5164835164835164, 0.2573099415204678, 0.44444444444444436], [0.7288888888888889, 0.5353159851301115, 0.357487922705314, 0.3915343915343915, 0.5026178010471204, 0.23391812865497075, 0.45454545454545453], [0.7219730941704036, 0.5530303030303031, 0.34782608695652173, 0.391304347826087, 0.507936507936508, 0.23809523809523808, 0.4533333333333333], [0.7232142857142857, 0.550185873605948, 0.3448275862068965, 0.40217391304347827, 0.5025641025641026, 0.2235294117647059, 0.4285714285714286], [0.7280898876404494, 0.537878787878788, 0.3546798029556651, 0.3957219251336898, 0.5106382978723403, 0.24705882352941178, 0.4503311258278146], [0.7203579418344519, 0.5185185185185185, 0.3517587939698492, 0.3915343915343915, 0.507936507936508, 0.2331288343558282, 0.45751633986928103]],
      [[0.7811158798283262, 0.5272727272727272, 0.19148936170212766, 0.372093023255814, 0.5, 0.09677419354838708, 0.45161290322580644], [0.7748917748917749, 0.5185185185185185, 0.22222222222222224, 0.380952380952381, 0.5000000000000001, 0.10714285714285715, 0.46153846153846156], [0.791578947368421, 0.5024154589371981, 0.2156862745098039, 0.3464566929133858, 0.47244094488188976, 0.1694915254237288, 0.45652173913043476], [0.7725321888412017, 0.5142857142857143, 0.22916666666666666, 0.4496124031007752, 0.48333333333333334, 0.10526315789473685, 0.4444444444444445], [0.7768240343347639, 0.5, 0.2745098039215686, 0.4028776978417266, 0.512, 0.1694915254237288, 0.4680851063829787], [0.7846481876332623, 0.5047619047619047, 0.2524271844660194, 0.37593984962406013, 0.4918032786885246, 0.13333333333333333, 0.45161290322580644], [0.7905982905982907, 0.5213270142180094, 0.22641509433962265, 0.36641221374045807, 0.47692307692307695, 0.10344827586206898, 0.4782608695652174], [0.7827956989247312, 0.5370370370370371, 0.22680412371134023, 0.3816793893129771, 0.5123966942148761, 0.16129032258064516, 0.45161290322580644], [0.7880085653104926, 0.5192307692307693, 0.2524271844660194, 0.36923076923076925, 0.5354330708661418, 0.16666666666666666, 0.4583333333333333], [0.7768240343347639, 0.509433962264151, 0.303030303030303, 0.35384615384615387, 0.4918032786885246, 0.13793103448275862, 0.46315789473684216]],
      [[0.7650727650727651, 0.5614035087719299, 0.42028985507246375, 0.49624060150375954, 0.4929577464788733, 0.3655913978494624, 0.4247787610619469], [0.7679671457905544, 0.5422222222222223, 0.43283582089552236, 0.536231884057971, 0.5034013605442177, 0.3829787234042553, 0.48275862068965525], [0.7628865979381443, 0.5652173913043479, 0.3909774436090226, 0.5, 0.5, 0.34951456310679613, 0.47058823529411764], [0.7761806981519507, 0.5486725663716815, 0.4233576642335766, 0.5217391304347826, 0.553191489361702, 0.3913043478260869, 0.49180327868852464], [0.7670103092783505, 0.5614035087719299, 0.4264705882352941, 0.5333333333333333, 0.5352112676056338, 0.3789473684210526, 0.47058823529411764], [0.7720739219712527, 0.5462555066079295, 0.40310077519379844, 0.49624060150375954, 0.547945205479452, 0.3404255319148936, 0.47457627118644075], [0.78099173553719, 0.5726872246696034, 0.40875912408759124, 0.5, 0.5103448275862069, 0.3478260869565218, 0.46956521739130436], [0.7711340206185568, 0.5617021276595745, 0.42028985507246375, 0.5035971223021583, 0.5390070921985816, 0.3695652173913044, 0.48695652173913045], [0.7748478701825559, 0.5244444444444444, 0.4, 0.4963503649635036, 0.5467625899280575, 0.3695652173913044, 0.4615384615384615], [0.7748478701825559, 0.5663716814159292, 0.3795620437956204, 0.5072463768115942, 0.5416666666666666, 0.3617021276595745, 0.48739495798319327]]],
     [[[0.7816593886462881, 0.5387755102040817, 0.43274853801169594, 0.41830065359477125, 0.5490196078431372, 0.4057971014492754, 0.5037037037037037], [0.7913043478260869, 0.5606694560669456, 0.4642857142857143, 0.4210526315789474, 0.5430463576158939, 0.3724137931034483, 0.5116279069767442], [0.7913978494623656, 0.540084388185654, 0.4444444444444444, 0.452054794520548, 0.5526315789473685, 0.3636363636363636, 0.46616541353383456], [0.7765726681127983, 0.5514403292181069, 0.45882352941176463, 0.44000000000000006, 0.5490196078431372, 0.36986301369863017, 0.4838709677419355], [0.7858719646799117, 0.5416666666666666, 0.4678362573099415, 0.44295302013422816, 0.5324675324675325, 0.3862068965517242, 0.49612403100775204], [0.7844827586206897, 0.5609756097560975, 0.4678362573099415, 0.4413793103448276, 0.5333333333333334, 0.3333333333333333, 0.4920634920634921], [0.7885462555066078, 0.55, 0.4615384615384615, 0.47682119205298007, 0.5466666666666666, 0.36986301369863017, 0.5038167938931297], [0.7868131868131868, 0.5371900826446282, 0.4615384615384615, 0.47133757961783446, 0.5369127516778524, 0.3776223776223776, 0.49612403100775204], [0.7903930131004367, 0.5454545454545455, 0.4508670520231214, 0.4459459459459459, 0.5599999999999999, 0.34285714285714286, 0.512], [0.7709251101321585, 0.5439330543933054, 0.45882352941176463, 0.4246575342465753, 0.5512820512820513, 0.36486486486486486, 0.515625]],
      [[0.7592190889370934, 0.5092592592592593, 0.3333333333333333, 0.37267080745341613, 0.5241379310344828, 0.19718309859154928, 0.4220183486238532], [0.7478260869565218, 0.5233644859813084, 0.2745098039215686, 0.41558441558441556, 0.496551724137931, 0.2191780821917808, 0.43636363636363634], [0.7586206896551725, 0.4880382775119617, 0.23300970873786409, 0.4121212121212121, 0.4964539007092198, 0.24657534246575344, 0.4485981308411215], [0.7444933920704846, 0.4928909952606635, 0.2828282828282828, 0.3870967741935484, 0.5106382978723405, 0.2564102564102564, 0.47058823529411764], [0.756043956043956, 0.5137614678899083, 0.2692307692307692, 0.3780487804878048, 0.47058823529411764, 0.26666666666666666, 0.46846846846846846], [0.7729257641921397, 0.5140186915887851, 0.2982456140350877, 0.45086705202312144, 0.4964539007092198, 0.21052631578947367, 0.47619047619047616], [0.7641921397379913, 0.5333333333333333, 0.2857142857142857, 0.39520958083832336, 0.48, 0.25974025974025977, 0.46846846846846846], [0.7538126361655775, 0.48309178743961356, 0.3364485981308411, 0.41975308641975306, 0.524822695035461, 0.10666666666666669, 0.4807692307692307], [0.7433628318584071, 0.4834123222748815, 0.3148148148148148, 0.43373493975903615, 0.4827586206896552, 0.16901408450704225, 0.45871559633027525], [0.7532467532467533, 0.4507042253521127, 0.3551401869158879, 0.3764705882352941, 0.4836601307189543, 0.2631578947368421, 0.42857142857142855]],
      [[0.7318087318087318, 0.5087719298245614, 0.32142857142857145, 0.3551401869158878, 0.42962962962962964, 0.12698412698412698, 0.4421052631578947], [0.7344398340248962, 0.497737556561086, 0.2424242424242424, 0.33962264150943394, 0.4888888888888889, 0.20338983050847456, 0.47058823529411764], [0.7464503042596349, 0.5155555555555555, 0.2095238095238095, 0.35051546391752575, 0.4496124031007752, 0.16666666666666666, 0.45098039215686275], [0.761904761904762, 0.5178571428571428, 0.2524271844660194, 0.396039603960396, 0.43410852713178294, 0.1694915254237288, 0.45161290322580644], [0.725, 0.5066666666666667, 0.26, 0.411214953271028, 0.4186046511627907, 0.1639344262295082, 0.4791666666666667], [0.7530364372469636, 0.4954128440366972, 0.20408163265306123, 0.36363636363636365, 0.4714285714285714, 0.25, 0.46464646464646464], [0.7603305785123968, 0.5089285714285714, 0.28865979381443296, 0.3888888888888889, 0.4411764705882353, 0.2, 0.4752475247524752], [0.7572016460905351, 0.5112107623318385, 0.26804123711340205, 0.34615384615384615, 0.4592592592592592, 0.3, 0.4554455445544554], [0.7484662576687117, 0.4910714285714286, 0.2127659574468085, 0.33027522935779813, 0.4852941176470589, 0.1694915254237288, 0.4693877551020408], [0.7302904564315352, 0.47393364928909953, 0.2745098039215686, 0.34615384615384615, 0.4444444444444444, 0.22222222222222218, 0.47058823529411764]]],
     [[[0.7619047619047619, 0.4634146341463415, 0.4347826086956521, 0.33333333333333337, 0.4055944055944056, 0.20779220779220778, 0.4554455445544554], [0.7619047619047619, 0.46808510638297873, 0.3878787878787879, 0.2476190476190476, 0.4460431654676259, 0.17500000000000002, 0.4571428571428572], [0.735930735930736, 0.5145228215767634, 0.35802469135802467, 0.25242718446601936, 0.43795620437956206, 0.17283950617283952, 0.43564356435643564], [0.7304347826086958, 0.4491525423728814, 0.43373493975903615, 0.3392857142857143, 0.4647887323943662, 0.24390243902439024, 0.44000000000000006], [0.7461368653421633, 0.46808510638297873, 0.4444444444444444, 0.37837837837837834, 0.4195804195804196, 0.23255813953488372, 0.4380952380952381], [0.7319148936170214, 0.4773662551440329, 0.4, 0.3247863247863248, 0.4460431654676259, 0.21176470588235294, 0.4901960784313726], [0.7473002159827216, 0.4396551724137931, 0.4125, 0.36666666666666664, 0.4316546762589928, 0.24390243902439024, 0.47619047619047616], [0.7531914893617021, 0.47500000000000003, 0.3928571428571428, 0.3185840707964602, 0.49635036496350365, 0.20779220779220778, 0.4752475247524752], [0.7516198704103673, 0.4890829694323144, 0.3544303797468354, 0.3243243243243243, 0.4028776978417266, 0.2285714285714286, 0.4444444444444445], [0.7450980392156863, 0.48535564853556495, 0.4177215189873418, 0.39655172413793105, 0.4689655172413793, 0.20253164556962028, 0.46601941747572817]],
      [[0.7759336099585062, 0.3431952662721894, 0.057971014492753624, 0.125, 0.2988505747126437, 0.0, 0.2727272727272727], [0.7868852459016393, 0.3536585365853659, 0.058823529411764705, 0.03225806451612903, 0.2298850574712644, 0.0, 0.25], [0.759753593429158, 0.37575757575757573, 0.02857142857142857, 0.03225806451612903, 0.3333333333333333, 0.0, 0.1694915254237288], [0.794238683127572, 0.38418079096045205, 0.028985507246376812, 0.125, 0.22727272727272727, 0.0, 0.36363636363636365], [0.7711340206185568, 0.38372093023255816, 0.0, 0.0625, 0.25, 0.0, 0.3582089552238805], [0.78099173553719, 0.38554216867469876, 0.029411764705882353, 0.033898305084745756, 0.2696629213483146, 0.0, 0.3225806451612903], [0.7858627858627858, 0.33540372670807456, 0.029850746268656716, 0.06451612903225806, 0.2765957446808511, 0.0, 0.33333333333333337], [0.7682672233820459, 0.36363636363636365, 0.05970149253731343, 0.15384615384615383, 0.3146067415730337, 0.0, 0.26229508196721313], [0.7633262260127931, 0.31788079470198677, 0.058823529411764705, 0.11764705882352941, 0.25287356321839083, 0.0, 0.2727272727272727], [0.775, 0.3647058823529412, 0.028985507246376812, 0.061538461538461535, 0.3260869565217391, 0.04347826086956522, 0.34375]],
      [[0.7370600414078675, 0.5022026431718062, 0.2524271844660194, 0.3551401869158878, 0.42424242424242425, 0.09836065573770493, 0.44680851063829785], [0.7577639751552795, 0.5333333333333333, 0.25925925925925924, 0.33333333333333337, 0.4776119402985075, 0.25806451612903225, 0.44897959183673475], [0.7515400410677618, 0.4954128440366972, 0.22000000000000003, 0.3125, 0.49230769230769234, 0.12698412698412698, 0.45098039215686275], [0.7551867219917013, 0.48868778280542985, 0.29411764705882354, 0.35294117647058826, 0.4159999999999999, 0.1694915254237288, 0.44897959183673475], [0.7364016736401674, 0.5, 0.25490196078431376, 0.40384615384615385, 0.46511627906976744, 0.18750000000000003, 0.4693877551020408], [0.7520325203252034, 0.502283105022831, 0.21782178217821785, 0.3551401869158878, 0.44285714285714284, 0.22580645161290322, 0.4554455445544554], [0.7714285714285715, 0.5045045045045046, 0.2708333333333333, 0.36190476190476184, 0.48062015503875966, 0.20338983050847456, 0.47058823529411764], [0.7714285714285715, 0.5089285714285714, 0.2476190476190476, 0.3333333333333333, 0.4511278195488722, 0.3125, 0.4285714285714286], [0.7555555555555556, 0.49339207048458145, 0.19148936170212766, 0.3703703703703704, 0.47692307692307695, 0.14285714285714285, 0.48936170212765956], [0.7418032786885245, 0.5114155251141553, 0.28, 0.3703703703703704, 0.43076923076923074, 0.18750000000000003, 0.4693877551020408]]]]

data = np.array(data)

In [0]:
data_df = pd.DataFrame(columns=["f1_score", "disorders", "data used", "classifier"])

best_classifiers = []
min_values = []
for most_common_disorder_i, most_common_disorder in enumerate(most_common_disorders):
    for data_configuration_i, data_configuration in enumerate(data_configurations):
        # find best classifier for the given problem
        median_values = np.mean(data[data_configuration_i, :, :, most_common_disorder_i], axis=1)

        best_classifier = np.argmax(median_values)
        best_classifier_name = classifiers[best_classifier]
        best_classifiers.append(best_classifier_name)
        min_values.append(np.min(data[data_configuration_i, best_classifier, :, most_common_disorder_i]))

        print('For', data_configuration, most_common_disorder, 'best classifier is', best_classifier_name, 'with a score of', median_values[best_classifier])

        for i in range(number_of_cross_val_runs):
            data_df = data_df.append({"f1_score": data[data_configuration_i, best_classifier, i, most_common_disorder_i],
                                      "disorders": most_common_disorder,
                                      "data used":  data_configuration,
                                      "classifier": best_classifier_name}, ignore_index=True)

In [0]:
fig, axes = plt.subplots(1, figsize=(18, 9))
sns.set()

sns.boxplot(x="disorders", y="f1_score", hue="data used", ax=axes, data=data_df, linewidth=2.5)
# sns.violinplot(x="disorders", y="f1_score", hue="data used", ax=axes, data=data_df, linewidth=2.5)

max_deviation = 0.3

pos = 0
add_pos = - max_deviation
total = 0
for clf, value in zip(best_classifiers, min_values):
    # print('position', pos + add_pos)
    axes.text(pos + add_pos - 0.025, value - 0.08, clf, horizontalalignment='left', color='black', weight='semibold', rotation=90)
    # print(total, len(data_configurations), total % len(data_configurations))
    
    if (total + 1) % len(data_configurations) == 0:
        pos += 1
        total = 0
        add_pos = - max_deviation
    else:
        add_pos += 2 * max_deviation / (len(data_configurations) - 1)
        total += 1

axes.set_ylim([0, 0.9])
if plot_only_data_from_intersection:
    axes.set_title('Use only intersection data')
    plt.savefig('intersetion_only.png')
else:
    axes.set_title('Train on all available data')
    plt.savefig('all_data.png')

most_common_disorders_labels = ['Attention-Deficit/\nHyperactivity Disorder', 'Anxiety Disorders', 'Specific Learning\nDisorder',
                                'Autism Spectrum\nDisorder', 'Disruptive', 'Communication Disorder', 'Depressive Disorders']
axes.set_xticklabels(most_common_disorders_labels)
classifiers_explanations = {"AE": "Autoencoder", "RF": "RandomForestClassifier", "SVM": "Support Vector Machine"}
text = "\n".join([k + ": " + v for k,v in zip(list(classifiers_explanations.keys()), list(classifiers_explanations.values()))])

axes.text(len(most_common_disorders) - 1 + 0.45, 0.68, text, horizontalalignment='right', color='black', weight='semibold', rotation=0)

axes.set_xlabel('Disorders', fontsize=15)
axes.set_ylabel('f1 score', fontsize=15)

plt.tight_layout()
plt.show()